In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,accuracy_score

In [2]:
data = {
    'x1': [0, 0, 1, 1],
    'x2': [0, 1, 0, 1],
    'y':  [0, 0, 0, 1]  # AND logic: only 1 AND 1 is 1, rest are 0
}

# Convert to a DataFrame
df = pd.DataFrame(data)
x1=df.pop('x1')
x2=df.pop('x2')
y=df.pop('y')

In [4]:
# a preceptron that functions as a AND gate
# z = w1x1+w2x2+b
# we will binary cross entropy loss function 0 or 1 classification
# Loss = -1/n sum(yi * log(y'i)+(1-yi)*log(1-y'i))
# dL/dy' = (-y/y' + (1-y)/(1-y'))
# a = sigmoid(z)
# a=y'
# dL/da = (-y/a + (1-y)/(1-a))
def L(y,a):
    n=len(y)
    return -1/n * np.sum(y * np.log(a) + (1 - y) * np.log(1 - a))
def ddLa(y,a):
    return -y/a + (1-y)/(1-a)

In [5]:
# a=sigmoid(z)
# da/dz = a(1-a)
def sigmoid(z):
    return 1 / (1 + np.exp(-z))
def ddaz(a):
    return a * (1-a)

In [15]:
# z = w1x1+w2x2+b
# dz/dw1 = x1
# dz/dw2 = x2
# dz/db = 1
# dL/dw1 = dL/da * da/dz * dz/dw1
# dL/dw2 = dL/da * da/dz * dz/dw2
# dL/db = dL/da * da/dz * dz/db
def ddLW1(y,a,x1):
    return ddLa(y,a)*ddaz(a)*x1
def ddLW2(y,a,x2):
    return ddLa(y,a)*ddaz(a)*x2
def ddLb(y,a):
    return ddLa(y,a)*ddaz(a)*1

In [6]:
#update equations are same
# w1 = w1 - learning_rate*dl/dw1
# w2 = w2 - learning_rate*dl/dw2
# b = b - learning_rate*dl/db
def forward(x1,x2,w1,w2,b):
    # find z
    z = w1 * x1 + w2 * x2 + b
    #print(z)
    a = sigmoid(z)
    #print(a)
    return z,a

In [7]:
def optimize(y,a,w1,w2,b,x1,x2,learning_rate):
    w1=w1-learning_rate*ddLW1(y,a,x1)
    w2=w2-learning_rate*ddLW2(y,a,x2)
    b=b-learning_rate*ddLb(y,a)
    return w1,w2,b

In [10]:
w1=np.random.rand() 
w2=np.random.rand() 
b=np.random.rand() 

In [16]:
steps=100
learning_rate=0.1

for i in range(steps):
    z,a=forward(x1,x2,w1,w2,b)
    w1,w2,b=optimize(y,a,w1,w2,b,x1,x2,learning_rate)

In [17]:
a

0    0.115877
1    0.058805
2    0.057303
3    0.971941
dtype: float64

In [18]:
#now we do it for football.csv file
df = pd.read_csv('football.csv')
df=df.drop('Team',axis=1)
df.corr()
y = df.pop('Points').to_numpy()
X = df.to_numpy() 
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
np.random.rand(X.shape[1]).shape

(6,)

In [26]:
X.T.shape

(6, 20)

In [ ]:
#6x20
#20x1
#6x1

In [27]:
class Perceptron:
    def __init__(self, input_shape, learning_rate):
        self.w = np.random.rand(input_shape)
        self.b = np.random.rand()
        self.learning_rate = learning_rate

    def forward(self, x):
        self.z = np.dot(x, self.w) + self.b
        self.a = self.relu(self.z)
        return self.a

    def predict(self, x):
        return self.forward(x)

    def optimize(self, x, y):
        n = len(y)
        dw = np.dot(x.T, (self.a - y)) / n
        db = np.sum(self.a - y) / n
        self.w -= self.learning_rate * dw
        self.b -= self.learning_rate * db

    @staticmethod
    def relu(z):
        return np.maximum(0,z)
    
    @staticmethod
    def sigmoid(z):
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))

In [28]:
p=Perceptron(X.shape[1],0.1)

In [39]:
steps=1000
learning_rate=0.1

for i in range(steps):
    p.forward(X_train)
    p.optimize(X_train,y_train)

In [40]:
y_pred=p.predict(X_test)
mean_squared_error(y_test, y_pred)

0.4991901395610187

In [42]:
df

,Passengerid,Age,Fare,Sex,sibsp,zero,zero.1,zero.2,zero.3,zero.4,...,zero.12,zero.13,zero.14,Pclass,zero.15,zero.16,Embarked,zero.17,zero.18,2urvived
0,1,22.0,7.2500,0,1,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
1,2,38.0,71.2833,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0.0,0,0,1
2,3,26.0,7.9250,1,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,1
3,4,35.0,53.1000,1,1,0,0,0,0,0,...,0,0,0,1,0,0,2.0,0,0,1
4,5,35.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,28.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
1305,1306,39.0,108.9000,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0.0,0,0,0
1306,1307,38.5,7.2500,0,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
1307,1308,28.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0


In [118]:
class Perceptron:
    def __init__(self, input_shape, learning_rate):
        self.w = np.random.rand(input_shape)
        self.b = np.random.rand()
        self.learning_rate = learning_rate

    def forward(self, x):
        self.z = np.dot(x, self.w) + self.b
        self.a = self.sigmoid(self.z)
        return self.a

    def predict(self, x):
        return self.forward(x)

    def optimize(self, x, y):
        n = len(y)
        dw = np.dot(x.T, (self.a - y)) / n
        db = np.sum(self.a - y) / n
        self.w -= self.learning_rate * dw
        self.b -= self.learning_rate * db

    @staticmethod
    def relu(z):
        return np.maximum(0,z)
    
    @staticmethod
    def sigmoid(z):
        return 1 / (1 + np.exp(-z))

In [119]:
#now we do it for football.csv file
df = pd.read_csv('./titatnic.csv')
df=df.drop('Passengerid',axis=1)
df=df.dropna(axis=0)
df.corr()
y = df.pop('2urvived').to_numpy()
df=df.drop(['zero','zero.1','zero.2','zero.3','zero.4','zero.5','zero.6','zero.7','zero.8','zero.9','zero.10','zero.11','zero.12','zero.13','zero.14','zero.15','zero.16','zero.17','zero.18'],axis=1)
X = df.to_numpy() 
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [127]:
p=Perceptron(X.shape[1],learning_rate=0.1)

In [129]:
steps=1000
learning_rate=0.0001

for i in range(steps):
    p.forward(X_train)
    p.optimize(X_train,y_train)
    pred=p.predict(X_test)
    y_pred = np.where(pred >= 0.5, 1, 0)
    print(accuracy_score(y_pred,y_test))

0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358778626
0.7900763358